<a href="https://colab.research.google.com/github/DuarteMagano/Hello-Quantum-World/blob/main/tutorial_students.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install qiskit==1.3.2 pylatexenc
%pip install qiskit-ibm-runtime==0.35.0
# this notebook was tested for qiskit version 1.3.2 and qiskit-ibm-runtime version 0.35.0
# be careful when running newer versions

# Quantum states

In [ ]:
import numpy as np

from qiskit.quantum_info import Statevector
from qiskit.visualization import plot_histogram, plot_bloch_multivector

# Quantum operations

In [ ]:
from qiskit.quantum_info import Operator
from qiskit.circuit.library import RXGate, CXGate

# Quantum circuits

In [ ]:
from qiskit import QuantumCircuit

# Sampler primitive

In [ ]:
from qiskit.primitives import StatevectorSampler
from qiskit import QuantumRegister, ClassicalRegister

# Multiple qubits

# Qiskit runtime

In [ ]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import SamplerV2

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from google.colab import userdata

In [ ]:
TOKEN = userdata.get('TOKEN')
service = QiskitRuntimeService(channel="ibm_quantum", token=TOKEN)

In [ ]:
backend = service.least_busy(operational=True, simulator=False)
print(f"{backend.name =}, {backend.num_qubits = }")

In [ ]:
pass_manager = generate_preset_pass_manager(
    backend=backend,
    optimization_level=1
)

In [ ]:
isa_qc = pass_manager.run(qc)
isa_qc.draw('mpl', idle_wires=False)

In [ ]:
sampler = SamplerV2(backend)

In [ ]:
job = sampler.run([isa_qc], shots=128)
job.job_id()

In [ ]:
job.status()

In [ ]:
results = job.result()[0]
counts = results.data.meas.get_counts()

In [ ]:
plot_histogram(counts)

# My first quantum algorithm

In [ ]:
def bv_function(s):
    """
    Create a Bernstein-Vazirani function from a string of 1s and 0s.
    """
    qc = QuantumCircuit(QuantumRegister(len(s), 'input'), QuantumRegister(1, 'output'))
    for index, bit in enumerate(reversed(s)):
        if bit == "1":
            qc.cx(index, len(s))
    return qc

In [ ]:
def bv_algo(function: QuantumCircuit):
    """
    Compiles a circuit for use in the Deutsch-Jozsa/Bernstein-Vazirani algorithm.
    """
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)
    qc.x(n)
    qc.barrier()
    qc.h(range(n + 1))
    qc.barrier()
    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()
    qc.barrier()
    qc.h(range(n))
    qc.barrier()
    qc.measure(range(n), range(n))
    return qc

# Variational quantum eigensolver

In [ ]:
from qiskit.circuit.library import EfficientSU2
from qiskit.primitives import StatevectorEstimator
from qiskit.quantum_info import SparsePauliOp

from scipy.optimize import minimize # SciPy minimizer routine
from scipy.linalg import eigh

import matplotlib.pyplot as plt

In [ ]:
hamiltonian = SparsePauliOp.from_list(
    [("YZ", 0.3980), ("ZI", -0.3980), ("ZZ", -0.0113), ("XX", 0.1810)]
)
hamiltonian.to_operator().draw("text")

In [ ]:
ansatz = EfficientSU2(hamiltonian.num_qubits)
ansatz.decompose().draw("mpl")

In [ ]:
cost_history = {
    "prev_vector": None,
    "iters": 0,
    "energy": [],
}

def cost_func(params, ansatz, hamiltonian, estimator):

    job = estimator.run([(ansatz, [hamiltonian], params)])
    result = job.result()[0]
    energy = result.data.evs.item()

    cost_history["iters"] += 1
    cost_history["prev_vector"] = params
    cost_history["energy"].append(energy)
    print(f"Iters. done: {cost_history['iters']} [Current cost: {energy}]")

    return energy

In [ ]:
params_init = 2 * np.pi * np.random.random(num_params)

res = minimize(
      cost_func,
      params_init,
      args=(ansatz, hamiltonian, estimator),
      method="cobyla",
      options = {"maxiter" : 500}
  )